# validate_images.ipynb

このノートブックは、**画像フォルダ**と**モデル（RT-DETR検出 / YOLO-seg）**を指定すると、

- 各画像の推論（Lens検出 → Lens内でRetina/Disc/Maculaセグメンテーション）
- 画像品質スコア（MBSS系 + Disc周辺）算出
- **足切り（retina_ratio と disc_ring_score）**を行い、足りなければ **retina_ratio閾値を段階的に緩和**
- 最終的に **Best画像（Top10/Top5）** を抽出

までを実行します。

> 想定入力: `image_dir` は `...\\画像` フォルダ（jpg/png等が入っている場所）


In [ ]:
import os
from pathlib import Path

import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
from ultralytics import RTDETR, YOLO

# -------------------- 入力パラメータ（ここだけ変更すればOK） --------------------

# 画像フォルダ（jpg/png等が入ったフォルダ）
image_dir = r"C:\\Users\\ykita\\ROP_AI_project\\ROP_project\\bestimage_validation\\1227\\画像"

# モデルパス
rtdetr_model_path = r"C:\\Users\\ykita\\ROP_AI_project\\ROP_project\\models\\rtdetr-l-1697_1703.pt"
yolo_seg_model_path = r"C:\\Users\\ykita\\ROP_AI_project\\ROP_project\\models\\yolo11n-seg_19movies.pt"

# 出力先（CSV / xlsx）
# - per-caseのCSVは `bestimage_validation/validation_results/` に保存（集計側と合わせる）
# - per-caseのBest画像Excelは `bestimage_validation/` に保存（任意）
output_root = r"C:\\Users\\ykita\\ROP_AI_project\\ROP_project\\bestimage_validation"

# Best候補数
top_k = 10   # 最終出力（Top10）
need_k = 5   # 研究用途で最低必要（Top5）

# 画像拡張子
image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff')

# YOLO入力幅（ノートブック内で固定）
YOLO_INPUT_WIDTH = 640

# -----------------------------------------------------------------------------

case_id = Path(image_dir).parent.name  # 例: ...\1227\画像 -> 1227
output_root = Path(output_root)
output_root.mkdir(parents=True, exist_ok=True)

# CSVは専用フォルダへ
validation_results_dir = output_root / "validation_results"
validation_results_dir.mkdir(parents=True, exist_ok=True)

output_csv_path = validation_results_dir / f"validation_results_{case_id}.csv"
output_best_xlsx_path = output_root / f"best_images_{case_id}.xlsx"

print("case_id:", case_id)
print("image_dir:", image_dir)
print("output_csv_path:", output_csv_path)
print("output_best_xlsx_path:", output_best_xlsx_path)


case_id: 1227
image_dir: C:\\Users\\ykita\\ROP_AI_project\\ROP_project\\bestimage_validation\\1227\\画像
output_csv_path: C:\Users\ykita\ROP_AI_project\ROP_project\bestimage_validation\validation_results_1227.csv
output_best_xlsx_path: C:\Users\ykita\ROP_AI_project\ROP_project\bestimage_validation\best_images_1227.xlsx


In [12]:
# ==================== 画像品質特徴量（MBSS） ====================

def to_gray_float(img_bgr_or_gray: np.ndarray) -> np.ndarray:
    """BGR/Gray いずれも float32 [0,1] グレースケールへ"""
    if img_bgr_or_gray.ndim == 3:
        gray = cv2.cvtColor(img_bgr_or_gray, cv2.COLOR_BGR2GRAY)
    else:
        gray = img_bgr_or_gray
    gray = gray.astype(np.float32)
    if gray.max() > 1.0:
        gray /= 255.0
    return gray


def laplacian_multi_var(gray01: np.ndarray, sigmas=(1.0, 2.0, 4.0), weights=(0.5, 0.3, 0.2)) -> float:
    """マルチスケール Laplacian 分散（重み付き和）"""
    vals = []
    for s, w in zip(sigmas, weights):
        ksize = int(6 * s + 1)
        if ksize % 2 == 0:
            ksize += 1
        blur = cv2.GaussianBlur(gray01, (ksize, ksize), s)
        lap = cv2.Laplacian(blur, cv2.CV_32F, ksize=3)
        vals.append(w * float(lap.var()))
    return float(np.sum(vals))


def fft_features(gray01: np.ndarray, high_freq_thresh=0.3) -> tuple:
    """FFT高周波エネルギー比とスペクトル重心（NumPy FFT版）"""
    h, w = gray01.shape

    wy = np.hanning(h).astype(np.float32)
    wx = np.hanning(w).astype(np.float32)
    window = np.outer(wy, wx)
    g = gray01 * window

    F = np.fft.fftshift(np.fft.fft2(g))
    mag2 = (np.abs(F) ** 2).astype(np.float64)

    cy, cx = h // 2, w // 2
    yy, xx = np.indices((h, w))
    ry = (yy - cy) / float(max(cy, 1))
    rx = (xx - cx) / float(max(cx, 1))
    r = np.sqrt(rx ** 2 + ry ** 2)
    r_norm = np.clip(r, 0, 1)

    total = mag2.sum() + 1e-12
    high_mask = r_norm > high_freq_thresh
    hf_ratio = float(mag2[high_mask].sum() / total)
    spec_centroid = float((r_norm * mag2).sum() / total)
    return hf_ratio, spec_centroid


from typing import Optional, Dict, Any


def grad_percentile(gray01: np.ndarray, p=90) -> float:
    gx = cv2.Sobel(gray01, cv2.CV_32F, 1, 0, ksize=3)
    gy = cv2.Sobel(gray01, cv2.CV_32F, 0, 1, ksize=3)
    mag = np.sqrt(gx ** 2 + gy ** 2)
    return float(np.percentile(mag, p))


def compute_mbss_components(img_bgr: np.ndarray, mask01: Optional[np.ndarray] = None) -> Dict[str, Any]:
    """Retina領域（mask）内のみで MBSS コンポーネントを算出"""
    gray = to_gray_float(img_bgr)

    if mask01 is not None:
        if mask01.shape != gray.shape:
            mask01 = cv2.resize(mask01.astype(np.uint8), (gray.shape[1], gray.shape[0]), interpolation=cv2.INTER_NEAREST)
        mask_bool = mask01 > 0
        if mask_bool.sum() < 100:
            return {"L_multi": None, "HF_ratio": None, "Spec_centroid": None, "Grad_p90": None}
        gray2 = gray.copy()
        gray2[~mask_bool] = 0.0
    else:
        gray2 = gray

    return {
        "L_multi": laplacian_multi_var(gray2),
        "HF_ratio": fft_features(gray2)[0],
        "Spec_centroid": fft_features(gray2)[1],
        "Grad_p90": grad_percentile(gray2),
    }


def compute_mbss_score(components: dict, stats: dict, weights=None) -> Optional[float]:
    """z-score正規化後、重み付き和でスコア化"""
    if any(components.get(k) is None for k in ["L_multi", "HF_ratio", "Spec_centroid", "Grad_p90"]):
        return None

    if weights is None:
        weights = {"L_multi": 0.35, "HF_ratio": 0.25, "Spec_centroid": 0.20, "Grad_p90": 0.20}

    score = 0.0
    for k, w in weights.items():
        x = float(components[k])
        m = float(stats[k]["mean"])
        s = float(stats[k]["std"]) + 1e-8
        z = (x - m) / s
        score += w * z
    return float(score)


# ==================== Disc周囲（core/ring）評価 ====================

def estimate_disc_center_radius(disc_mask01: np.ndarray):
    """discマスクから中心(cx,cy)と代表半径Rを推定"""
    m = disc_mask01.astype(np.uint8)
    if m.max() > 1:
        m = (m > 0).astype(np.uint8)

    num_labels, labels = cv2.connectedComponents(m)
    if num_labels > 1:
        areas = [(labels == i).sum() for i in range(1, num_labels)]
        main_label = int(np.argmax(areas) + 1)
        m = (labels == main_label).astype(np.uint8)

    M = cv2.moments(m)
    if M["m00"] == 0:
        return None

    cx = M["m10"] / M["m00"]
    cy = M["m01"] / M["m00"]
    area = float(m.sum())
    R = float(np.sqrt(area / np.pi))
    return cx, cy, R


def make_disc_rois(shape_hw, cx, cy, R, inner_ratio=0.6, outer_ratio=1.2):
    h, w = shape_hw
    yy, xx = np.indices((h, w))
    dist = np.sqrt((xx - cx) ** 2 + (yy - cy) ** 2)
    core = dist < (inner_ratio * R)
    ring = (dist >= (inner_ratio * R)) & (dist < (outer_ratio * R))
    return core.astype(np.uint8), ring.astype(np.uint8)


def laplacian_multi_var_masked(gray01: np.ndarray, mask01: np.ndarray, sigmas=(1.0, 2.0, 4.0), weights=(0.5, 0.3, 0.2)) -> float:
    mask_bool = mask01.astype(bool)
    if mask_bool.sum() < 50:
        return 0.0

    vals = []
    for s, w in zip(sigmas, weights):
        ksize = int(6 * s + 1)
        if ksize % 2 == 0:
            ksize += 1
        blur = cv2.GaussianBlur(gray01, (ksize, ksize), s)
        lap = cv2.Laplacian(blur, cv2.CV_32F, ksize=3)
        roi = lap[mask_bool]
        if roi.size == 0:
            continue
        vals.append(w * float(roi.var()))
    return float(np.sum(vals)) if vals else 0.0


def compute_disc_sharpness_components(img_bgr: np.ndarray, disc_mask01: np.ndarray):
    """disc中心(core)と周辺(ring)の L_multi を返す"""
    gray = to_gray_float(img_bgr)
    est = estimate_disc_center_radius(disc_mask01)
    if est is None:
        return None, None

    cx, cy, R = est
    core_mask, ring_mask = make_disc_rois(gray.shape, cx, cy, R)

    if core_mask.sum() < 50 or ring_mask.sum() < 50:
        return None, None

    L_core = laplacian_multi_var_masked(gray, core_mask)
    L_ring = laplacian_multi_var_masked(gray, ring_mask)
    return L_core, L_ring


In [13]:
from typing import Optional


def process_one_image(image_path: str, detection_model, segmentation_model) -> Optional[dict]:
    """1枚の画像に対して推論 + 特徴量を算出"""
    image = cv2.imread(image_path)
    if image is None:
        return None

    # --- Stage 1: RT-DETRでLens bbox検出（cls=0想定） ---
    det_results = detection_model(image, verbose=False)
    lens_bbox_xyxy = None
    for r in det_results:
        if r.boxes is None or len(r.boxes) == 0:
            continue
        for box in r.boxes:
            if int(box.cls) == 0:
                lens_bbox_xyxy = box.xyxy[0].cpu().numpy()
                break
        if lens_bbox_xyxy is not None:
            break

    if lens_bbox_xyxy is None:
        return {
            'image_path': image_path,
            'lens_detected': False,
            'lens_area': 0,
            'retina_area': 0,
            'retina_ratio': 0.0,
            'disc_detected': False,
            'macula_detected': False,
            'mbss_L_multi': None,
            'mbss_HF_ratio': None,
            'mbss_Spec_centroid': None,
            'mbss_Grad_p90': None,
            'disc_core_L_multi': None,
            'disc_ring_L_multi': None,
        }

    x1, y1, x2, y2 = [int(c) for c in lens_bbox_xyxy]
    cropped = image[y1:y2, x1:x2]
    if cropped.size == 0:
        return {
            'image_path': image_path,
            'lens_detected': True,
            'lens_area': 0,
            'retina_area': 0,
            'retina_ratio': 0.0,
            'disc_detected': False,
            'macula_detected': False,
            'mbss_L_multi': None,
            'mbss_HF_ratio': None,
            'mbss_Spec_centroid': None,
            'mbss_Grad_p90': None,
            'disc_core_L_multi': None,
            'disc_ring_L_multi': None,
        }

    # --- Lens内での円形マスク（レンズ外を灰色にする） ---
    orig_h, orig_w = cropped.shape[:2]
    center_x = orig_w // 2
    center_y = orig_h // 2
    diameter = (orig_w + orig_h) / 2
    radius = int(diameter / 2)

    circle_mask = np.zeros((orig_h, orig_w), dtype=np.uint8)
    cv2.circle(circle_mask, (center_x, center_y), radius, 255, -1)

    masked_cropped = cropped.copy()
    masked_cropped[circle_mask == 0] = (114, 114, 114)

    lens_area = int((circle_mask > 0).sum())

    # --- Stage 2: YOLO-seg ---
    aspect_ratio = orig_h / max(orig_w, 1)
    yolo_h = int(YOLO_INPUT_WIDTH * aspect_ratio)
    yolo_input = cv2.resize(masked_cropped, (YOLO_INPUT_WIDTH, yolo_h), interpolation=cv2.INTER_AREA)

    seg_results = segmentation_model(yolo_input, verbose=False, retina_masks=True)

    retina_area = 0
    disc_detected = False
    macula_detected = False

    retina_mask_crop = None
    disc_mask_crop = None

    if seg_results and seg_results[0].masks is not None:
        r0 = seg_results[0]
        masks = r0.masks.data.cpu().numpy()
        classes = r0.boxes.cls.cpu().numpy().astype(int)

        for mask_data, cls_id in zip(masks, classes):
            # mask_data: (H', W') 0..1
            mask_resized = cv2.resize(mask_data, (orig_w, orig_h), interpolation=cv2.INTER_LINEAR)
            mask_bin = (mask_resized > 0.5) & (circle_mask > 0)

            if cls_id == 0:  # Fundus/Retina
                retina_area = int(mask_bin.sum())
                retina_mask_crop = (mask_bin.astype(np.uint8) * 255)
            elif cls_id == 1:  # Disc
                disc_detected = True
                disc_mask_crop = (mask_bin.astype(np.uint8) * 255)
            elif cls_id == 2:  # Macula
                macula_detected = True

    retina_ratio = (retina_area / lens_area * 100.0) if lens_area > 0 else 0.0

    # --- MBSS（Retina領域内） ---
    if retina_mask_crop is not None:
        mb = compute_mbss_components(cropped, mask01=retina_mask_crop)
    else:
        mb = {"L_multi": None, "HF_ratio": None, "Spec_centroid": None, "Grad_p90": None}

    # --- Disc周囲（core/ring） ---
    disc_core_L_multi = None
    disc_ring_L_multi = None
    if disc_mask_crop is not None:
        disc_core_L_multi, disc_ring_L_multi = compute_disc_sharpness_components(cropped, disc_mask_crop)

    return {
        'image_path': image_path,
        'lens_detected': True,
        'lens_area': lens_area,
        'retina_area': retina_area,
        'retina_ratio': round(float(retina_ratio), 2),
        'disc_detected': bool(disc_detected),
        'macula_detected': bool(macula_detected),
        'mbss_L_multi': mb['L_multi'],
        'mbss_HF_ratio': mb['HF_ratio'],
        'mbss_Spec_centroid': mb['Spec_centroid'],
        'mbss_Grad_p90': mb['Grad_p90'],
        'disc_core_L_multi': disc_core_L_multi,
        'disc_ring_L_multi': disc_ring_L_multi,
    }


In [14]:
# ==================== 推論実行（フォルダ内の全画像） ====================

# 重要: 先に「画像品質特徴量（MBSS）」「process_one_image」セルを実行して、
# compute_mbss_components 等が定義されている必要があります。
required_funcs = [
    'compute_mbss_components',
    'compute_disc_sharpness_components',
    'process_one_image',
]
missing = [n for n in required_funcs if n not in globals()]
if missing:
    raise RuntimeError(
        "必要な関数が未定義です（実行順を確認）: " + ", ".join(missing) +
        "\n→ 上から順にセルを実行（またはKernel再起動→全セル実行）してください。"
    )

print("モデルを読み込んでいます...")
detection_model = RTDETR(rtdetr_model_path)
segmentation_model = YOLO(yolo_seg_model_path)

if torch.cuda.is_available():
    detection_model.to('cuda')
    segmentation_model.to('cuda')
    print("CUDAを使用します")
else:
    print("CPUを使用します")

image_files = sorted([f for f in os.listdir(image_dir) if f.lower().endswith(image_extensions)])
print("画像枚数:", len(image_files))

results = []
for fname in tqdm(image_files, desc="画像を処理中"):
    p = os.path.join(image_dir, fname)
    try:
        r = process_one_image(p, detection_model, segmentation_model)
        if r is None:
            continue
        r['image_name'] = fname
        r['image_id'] = case_id
        results.append(r)
    except Exception as e:
        print(f"エラー: {fname}: {e}")

if not results:
    raise RuntimeError("処理結果が0件です。image_dir/モデル/依存関係を確認してください")

# DataFrame化
_df = pd.DataFrame(results)

# -------------------- スコア算出（ID内でz-score） --------------------

# MBSS stats（None除外）
mb_cols = ['mbss_L_multi', 'mbss_HF_ratio', 'mbss_Spec_centroid', 'mbss_Grad_p90']
stats = {}
for c in mb_cols:
    vals = _df[c].dropna().astype(float)
    key = c.replace('mbss_', '')
    if len(vals) > 0 and float(vals.std()) > 0:
        stats[key] = {"mean": float(vals.mean()), "std": float(vals.std())}
    elif len(vals) > 0:
        stats[key] = {"mean": float(vals.mean()), "std": 1.0}

# MBSS score
mbss_scores = []
for _, row in _df.iterrows():
    comps = {
        "L_multi": row.get('mbss_L_multi'),
        "HF_ratio": row.get('mbss_HF_ratio'),
        "Spec_centroid": row.get('mbss_Spec_centroid'),
        "Grad_p90": row.get('mbss_Grad_p90'),
    }
    if set(stats.keys()) == {"L_multi", "HF_ratio", "Spec_centroid", "Grad_p90"}:
        mbss_scores.append(compute_mbss_score(comps, stats=stats))
    else:
        mbss_scores.append(None)
_df['mbss_score'] = mbss_scores

# Disc core/ring score（z-score）
for col_l, col_s in [('disc_core_L_multi', 'disc_core_score'), ('disc_ring_L_multi', 'disc_ring_score')]:
    vals = _df[col_l].dropna().astype(float)
    if len(vals) > 1 and float(vals.std()) > 0:
        m, s = float(vals.mean()), float(vals.std())
    elif len(vals) > 0:
        m, s = float(vals.mean()), 1.0
    else:
        m, s = 0.0, 1.0

    scores = []
    for v in _df[col_l]:
        if v is None or pd.isna(v):
            scores.append(None)
        else:
            scores.append((float(v) - m) / (s + 1e-8))
    _df[col_s] = scores

# 保存（CSV）
columns_order = [
    'image_id','image_name','image_path','lens_detected','lens_area','retina_area','retina_ratio',
    'disc_detected','macula_detected',
    'mbss_L_multi','mbss_HF_ratio','mbss_Spec_centroid','mbss_Grad_p90','mbss_score',
    'disc_core_L_multi','disc_core_score','disc_ring_L_multi','disc_ring_score'
]
columns_order = [c for c in columns_order if c in _df.columns]
_df = _df[columns_order]

_df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')
print("保存しました:", output_csv_path)
print("処理件数:", len(_df))

_df.head()


モデルを読み込んでいます...
CUDAを使用します
画像枚数: 659


画像を処理中: 100%|██████████| 659/659 [02:14<00:00,  4.91it/s]


保存しました: C:\Users\ykita\ROP_AI_project\ROP_project\bestimage_validation\validation_results_1227.csv
処理件数: 659


,image_id,image_name,image_path,lens_detected,lens_area,retina_area,retina_ratio,disc_detected,macula_detected,mbss_L_multi,mbss_HF_ratio,mbss_Spec_centroid,mbss_Grad_p90,mbss_score,disc_core_L_multi,disc_core_score,disc_ring_L_multi,disc_ring_score
0,1227,IMG_1227_0005.jpg,C:\\Users\\ykita\\ROP_AI_project\\ROP_project\...,False,0,0,0.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1227,IMG_1227_0010.jpg,C:\\Users\\ykita\\ROP_AI_project\\ROP_project\...,True,239197,0,0.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1227,IMG_1227_0015.jpg,C:\\Users\\ykita\\ROP_AI_project\\ROP_project\...,False,0,0,0.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1227,IMG_1227_0020.jpg,C:\\Users\\ykita\\ROP_AI_project\\ROP_project\...,True,570975,0,0.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1227,IMG_1227_0025.jpg,C:\\Users\\ykita\\ROP_AI_project\\ROP_project\...,True,431528,0,0.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# ==================== Best画像選定（足切り + retina_ratio閾値を段階的に緩和） ====================

# 有効データのみ
valid = _df[(_df['lens_detected'] == True) & (_df['retina_ratio'] > 0)].copy()

if len(valid) == 0:
    raise RuntimeError("lens_detected=True かつ retina_ratio>0 のデータがありません")

# disc_ring_score が十分に計算できている場合のみ足切りに使う
ring_nonnull = valid['disc_ring_score'].notna().sum() if 'disc_ring_score' in valid.columns else 0
use_disc_ring_filter = ring_nonnull >= need_k

disc_ring_median = None
if use_disc_ring_filter:
    disc_ring_median = float(valid['disc_ring_score'].median())
    print(f"disc_ring_score中央値(全体): {disc_ring_median:.3f}")
else:
    print(f"disc_ring_score が有効な件数が少ないため（{ring_nonnull}件）、disc_ring_score足切りは使いません")

# 候補数を確保するため、retina_ratio閾値を緩めていく
percentiles = [0.90, 0.80, 0.70, 0.60, 0.50, 0.40, 0.30, 0.20, 0.10, 0.0]

candidates = pd.DataFrame()
for p in percentiles:
    thr = float(valid['retina_ratio'].quantile(p))
    step1 = valid[valid['retina_ratio'] >= thr].copy()

    if use_disc_ring_filter:
        step2 = step1[(step1['disc_ring_score'].notna()) & (step1['disc_ring_score'] >= disc_ring_median)].copy()
    else:
        step2 = step1

    if len(step2) >= top_k:
        candidates = step2
        print(f"候補: {len(candidates)}件（retina_ratio閾値={thr:.2f}, p={p:.2f}）")
        break

    if p == 0.0:
        candidates = step2
        print(f"候補: {len(candidates)}件（最大限緩和: retina_ratio閾値={thr:.2f}）")

if len(candidates) == 0:
    raise RuntimeError("候補が0件です（閾値緩和後も）")

# ランキング（rank_sum = mbss_rank + disc_core_rank）
# NaNはbottomに落とす
if 'mbss_score' not in candidates.columns:
    candidates['mbss_score'] = np.nan
if 'disc_core_score' not in candidates.columns:
    candidates['disc_core_score'] = np.nan

# 追加要件:
# retina_ratioの閾値を緩めて候補を増やした場合でも、
# 「より厳しい閾値（高いパーセンタイル）を満たす画像」を常に上位にする。
# → 各画像が満たす最大パーセンタイル（retina_tier）を付与して tier優先でソート。
thr_by_p = {p: float(valid['retina_ratio'].quantile(p)) for p in percentiles}
candidates = candidates.copy()
candidates['retina_tier'] = 0.0
for p in sorted(percentiles):  # low -> high (high wins)
    thr = thr_by_p[p]
    candidates.loc[candidates['retina_ratio'] >= thr, 'retina_tier'] = float(p)

candidates['mbss_rank'] = candidates['mbss_score'].rank(ascending=False, method='min', na_option='bottom')
candidates['disc_core_rank'] = candidates['disc_core_score'].rank(ascending=False, method='min', na_option='bottom')
candidates['rank_sum'] = candidates['mbss_rank'] + candidates['disc_core_rank']

# Sort priority:
# 1) retina_tier (desc): 厳しいretina_ratio閾値を満たすほど上位
# 2) rank_sum (asc)
# 3) mbss_score (desc) tie-break
ranked = candidates.sort_values(by=['retina_tier', 'rank_sum', 'mbss_score'], ascending=[False, True, False])

best_top10 = ranked.head(top_k).copy()
best_top5 = ranked.head(need_k).copy()

print("\n=== Best Top10 (basename) ===")
print(best_top10['image_name'].tolist())

print("\n=== Best Top5 (basename) ===")
print(best_top5['image_name'].tolist())

# Excel保存
out_cols = ['image_id','image_name','image_path','retina_ratio','mbss_score','disc_core_score','disc_ring_score','rank_sum','mbss_rank','disc_core_rank']
out_cols = [c for c in out_cols if c in best_top10.columns]

out_df = best_top10[out_cols].copy()
try:
    out_df.to_excel(output_best_xlsx_path, index=False)
    print("\n保存しました:", output_best_xlsx_path)
except Exception as e:
    print("\nExcel出力に失敗しました（openpyxl未導入の可能性）:", e)
    # 代替: CSV
    alt_csv = output_best_xlsx_path.with_suffix('.csv')
    out_df.to_csv(alt_csv, index=False, encoding='utf-8-sig')
    print("代替でCSV保存しました:", alt_csv)

out_df


disc_ring_score中央値(全体): -0.302
候補: 16件（retina_ratio閾値=81.78, p=0.90）

=== Best Top10 (basename) ===
['IMG_1227_1095.jpg', 'IMG_1227_1100.jpg', 'IMG_1227_1110.jpg', 'IMG_1227_1090.jpg', 'IMG_1227_1120.jpg', 'IMG_1227_1115.jpg', 'IMG_1227_1170.jpg', 'IMG_1227_1160.jpg', 'IMG_1227_1165.jpg', 'IMG_1227_1125.jpg']

=== Best Top5 (basename) ===
['IMG_1227_1095.jpg', 'IMG_1227_1100.jpg', 'IMG_1227_1110.jpg', 'IMG_1227_1090.jpg', 'IMG_1227_1120.jpg']

保存しました: C:\Users\ykita\ROP_AI_project\ROP_project\bestimage_validation\best_images_1227.xlsx


,image_id,image_name,image_path,retina_ratio,mbss_score,disc_core_score,disc_ring_score,rank_sum,mbss_rank,disc_core_rank
218,1227,IMG_1227_1095.jpg,C:\\Users\\ykita\\ROP_AI_project\\ROP_project\...,87.32,0.696499,0.080180,0.228799,5.0,3.0,2.0
219,1227,IMG_1227_1100.jpg,C:\\Users\\ykita\\ROP_AI_project\\ROP_project\...,86.70,0.657989,0.121518,0.657001,5.0,4.0,1.0
221,1227,IMG_1227_1110.jpg,C:\\Users\\ykita\\ROP_AI_project\\ROP_project\...,84.97,0.630847,-0.002132,0.697880,11.0,6.0,5.0
217,1227,IMG_1227_1090.jpg,C:\\Users\\ykita\\ROP_AI_project\\ROP_project\...,87.17,0.588459,0.079458,0.157245,11.0,8.0,3.0
223,1227,IMG_1227_1120.jpg,C:\\Users\\ykita\\ROP_AI_project\\ROP_project\...,83.63,0.593899,-0.002660,0.568586,13.0,7.0,6.0
222,1227,IMG_1227_1115.jpg,C:\\Users\\ykita\\ROP_AI_project\\ROP_project\...,82.88,0.544702,0.044670,0.433587,14.0,10.0,4.0
233,1227,IMG_1227_1170.jpg,C:\\Users\\ykita\\ROP_AI_project\\ROP_project\...,86.76,0.768762,-0.252953,-0.272698,15.0,1.0,14.0
231,1227,IMG_1227_1160.jpg,C:\\Users\\ykita\\ROP_AI_project\\ROP_project\...,85.04,0.716976,-0.214537,-0.173962,15.0,2.0,13.0
232,1227,IMG_1227_1165.jpg,C:\\Users\\ykita\\ROP_AI_project\\ROP_project\...,88.39,0.656938,-0.208131,-0.234979,16.0,5.0,11.0
224,1227,IMG_1227_1125.jpg,C:\\Users\\ykita\\ROP_AI_project\\ROP_project\...,84.17,0.511005,-0.032642,-0.126281,18.0,11.0,7.0
